In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/transactions_cleaned.csv')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df.sort_values('Transaction Date')
df['Price'] = df['Price_0_days']
df

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,Transaction Type,Value Range,Price,Transaction ID,Price_0_days,Approx Transaction Amount,Approx Share Count
0,Tom Carper,Democrat,Senate,DE,Valero Energy Corp,VLO,2024-05-01,2024-04-11,19,Spouse,SELL,1K–15K,177.039993,10000062280,177.039993,8000.0,45.0
1,Tom Carper,Democrat,Senate,DE,Enbridge Inc,ENB,2024-05-01,2024-04-11,19,Spouse,BUY,1K–15K,34.549999,10000062281,34.549999,8000.0,231.0
2,Dan Newhouse,Republican,House,WA,Accenture PLC,ACN,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,324.399994,20003772725,324.399994,8000.0,24.0
3,Dan Newhouse,Republican,House,WA,Analog Devices Inc,ADI,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,196.330002,20003772726,196.330002,8000.0,40.0
4,Dan Newhouse,Republican,House,WA,Booking Holdings Inc,BKNG,2024-04-26,2024-04-10,15,Spouse,SELL,1K–15K,3586.129883,20003772728,3586.129883,8000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31584,Alan Lowenthal,Democrat,House,CA,Americold Realty Trust,COLD,2021-03-24,2021-03-15,8,Spouse,SELL,15K–50K,37.130001,20001621840,37.130001,32500.0,875.0
31585,Mark Green,Republican,House,TN,NGL Energy Partners LP,NGL,2021-03-21,2021-03-17,3,Undisclosed,BUY,15K–50K,2.340000,20001620394,2.340000,32500.0,13888.0
31586,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Spouse,SELL,15K–50K,70.160004,20001621844,70.160004,32500.0,463.0
31587,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Undisclosed,SELL,1K–15K,70.160004,20001621847,70.160004,8000.0,114.0


In [2]:
def calculate_annualized_percentage_profit_with_ids(df):
    results = []

    grouped = df.groupby(['Politician Name', 'Ticker'])

    for (name, ticker), group in grouped:
        transactions = group.sort_values('Transaction Date').to_dict('records')

        buy_prices = []
        sell_prices = []
        buy_ids = []
        sell_ids = []
        buy_amounts = []
        sell_amounts = []
        buy_dates = []
        sell_dates = []
        buying = True

        for transaction in transactions:
            date = transaction['Transaction Date']
            if buying and transaction['Transaction Type'] == 'BUY':
                #adding to the current buy cycle
                buy_prices.append(transaction['Price'])
                buy_ids.append(transaction['Transaction ID'])
                buy_amounts.append(transaction['Approx Transaction Amount'])
                buy_dates.append(date)
            elif buying and transaction['Transaction Type'] == 'SELL' and len(buy_prices):
                #transition from buying to selling
                buying = False
                sell_prices.append(transaction['Price'])
                sell_ids.append(transaction['Transaction ID'])
                sell_amounts.append(transaction['Approx Transaction Amount'])
                sell_dates.append(date)
            elif not buying and transaction['Transaction Type'] == 'SELL':
                #adding to the current sell cycle
                sell_prices.append(transaction['Price'])
                sell_ids.append(transaction['Transaction ID'])
                sell_amounts.append(transaction['Approx Transaction Amount'])
                sell_dates.append(date)
            elif not buying and transaction['Transaction Type'] == 'BUY':
                #close the previous sell cycle before starting a new buy cycle
                if buy_prices and sell_prices:
                    cycle_results = process_cycle(name, ticker, buy_prices, sell_prices, buy_ids, sell_ids, buy_amounts, sell_amounts, buy_dates, sell_dates)
                    results.extend(cycle_results)

                buy_prices, sell_prices, buy_ids, sell_ids, buy_amounts, sell_amounts, buy_dates, sell_dates = [], [], [], [], [], [], [], []

                buying = True
                buy_prices.append(transaction['Price'])
                buy_ids.append(transaction['Transaction ID'])
                buy_amounts.append(transaction['Approx Transaction Amount'])
                buy_dates.append(date)

        if buy_prices and sell_prices:
            cycle_results = process_cycle(name, ticker, buy_prices, sell_prices, buy_ids, sell_ids, buy_amounts, sell_amounts, buy_dates, sell_dates)
            results.extend(cycle_results)

    return pd.DataFrame(results)

def process_cycle(name, ticker, buy_prices, sell_prices, buy_ids, sell_ids, buy_amounts, sell_amounts, buy_dates, sell_dates):
    if not buy_dates or not sell_dates:
        return []

    first_buy_date = min(buy_dates) if buy_dates else None
    last_sell_date = max(sell_dates) if sell_dates else None
    days_between = (last_sell_date - first_buy_date).days if first_buy_date and last_sell_date else 0

    if days_between > 0 and buy_prices and sell_prices:
        avg_buy_price = sum(buy_prices) / len(buy_prices)
        avg_sell_price = sum(sell_prices) / len(sell_prices)
        percentage_profit = (avg_sell_price - avg_buy_price) / avg_buy_price * 100
        annualized_profit = (percentage_profit / days_between) * 365
        return [{
            'Politician': name,
            'Ticker': ticker,
            'Annualized Percentage Profit': annualized_profit,
            'Purchase Transaction IDs': buy_ids,
            'Sell Transaction IDs': sell_ids,
            'Total Purchase Amount': sum(buy_amounts),
            'Total Sell Amount': sum(sell_amounts),
            'Days Between': days_between
        }]
    return []

trades = calculate_annualized_percentage_profit_with_ids(df)

In [3]:
trades.sort_values('Annualized Percentage Profit', ascending=False)

,Politician,Ticker,Annualized Percentage Profit,Purchase Transaction IDs,Sell Transaction IDs,Total Purchase Amount,Total Sell Amount,Days Between
1386,Marie Newman,PTON,3161.461424,[20003694412],[20003694413],8000.0,8000.0,2
722,Jim Banks,RBLX,1960.844078,[20003708653],"[20003708651, 20003708654]",32500.0,40500.0,7
5158,Tommy Tuberville,PYPL,1630.887975,[10000060480],[10000060482],175000.0,175000.0,1
3020,Ro Khanna,GOOGL,1233.239019,[20003761778],"[20003762017, 20003761597]",32500.0,16000.0,2
2108,Ro Khanna,ARE,1200.598085,[20003768918],[20003769728],8000.0,8000.0,3
...,...,...,...,...,...,...,...,...
4812,Ro Khanna,ZM,-1095.760885,[20003684224],[20003684253],8000.0,8000.0,5
3445,Ro Khanna,LULU,-1247.404265,[20003772192],[20003772110],8000.0,8000.0,5
5057,Tom Suozzi,UPST,-1909.892368,"[20003723764, 20003723765]","[20003723763, 20003723766]",65000.0,16000.0,11
3718,Ro Khanna,MTCH,-2354.066985,[20003757234],[20003757516],8000.0,32500.0,1


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
file_path = '/content/drive/MyDrive/Politician Trades/trades.csv'
trades.to_csv(file_path, index=False)